# UGrid Network1d basics

This is the basic introduction for using the `ugridpy` library.

`ugridpy` can be used for reading and writing network1d. 

At the very beginning, the necessary libraries have to be imported.

In [5]:
from ugrid import UGrid, UGridNetwork1D
from ugrid.version import __version__
__version__

'0.9.0'

Other imports

In [6]:
import numpy as np

## Network1d reading

In [7]:
with UGrid("./data_examples/AllUGridEntities.nc", "r") as ug:
    # 1. Count the number of network topologies
    num_network_topologies = ug.network1d_get_num_topologies()
    # 2. Get the data of the last network in the list
    network1d = ug.network1d_get(num_network_topologies - 1)

### Print some data

In [8]:
print(f"Network 1D name: {network1d.name}")
print(f"Network 1D num branch geometry nodes: {network1d.num_edge_geometry_nodes}")
print(f"Network 1D branch nodes: {network1d.edge_node}")
print(f"Network 1D node x: {network1d.node_x}")
print(f"Network 1D node y: {network1d.node_y}")
print(f"Network 1D geometry nodes x: {network1d.geometry_nodes_x}")
print(f"Network 1D geometry nodes y: {network1d.geometry_nodes_y}")
print(f"Network 1D node name id: {network1d.node_id}")
print(f"Network 1D node name long: {network1d.node_long_name}")
print(f"Network 1D is spherical: {network1d.is_spherical}")

Network 1D name: network
Network 1D num branch geometry nodes: [-1330765608]
Network 1D branch nodes: [0 1]
Network 1D node x: [293.78 538.89]
Network 1D node y: [ 27.48 956.75]
Network 1D geometry nodes x: [293.78 278.97 265.31 254.17 247.44 248.3  259.58 282.24 314.61 354.44
 398.94 445.   490.6  532.84 566.64 589.08 600.72 603.53 599.27 590.05
 577.56 562.97 547.12 530.67 538.89]
Network 1D geometry nodes y: [ 27.48  74.87 122.59 170.96 220.12 269.67 317.89 361.93 399.39 428.84
 450.76 469.28 488.89 514.78 550.83 594.93 643.09 692.6  742.02 790.79
 838.83 886.28 933.33 980.17 956.75]
Network 1D node name id: ['nodesids', 'nodesids']
Network 1D node name long: ['nodeslongNames', 'nodeslongNames']
Network 1D is spherical: False


## Network1d writing

### Define Network1d arrays

In [9]:
node_x = np.array([293.78, 538.89], dtype=np.double)
node_y = np.array([27.48, 956.75], dtype=np.double)
branch_node = np.array([0, 1], dtype=np.int32)
branch_length = np.array([1165.29], dtype=np.double)
branch_order = np.array([0], dtype=np.int32)

geometry_nodes_x = np.array(
    [
        293.78,
        278.97,
        265.31,
        254.17,
        247.44,
        248.3,
        259.58,
        282.24,
        314.61,
        354.44,
        398.94,
        445,
        490.6,
        532.84,
        566.64,
        589.08,
        600.72,
        603.53,
        599.27,
        590.05,
        577.56,
        562.97,
        547.12,
        530.67,
        538.89,
    ],
    dtype=np.double,
)

geometry_nodes_y = np.array(
    [
        27.48,
        74.87,
        122.59,
        170.96,
        220.12,
        269.67,
        317.89,
        361.93,
        399.39,
        428.84,
        450.76,
        469.28,
        488.89,
        514.78,
        550.83,
        594.93,
        643.09,
        692.6,
        742.02,
        790.79,
        838.83,
        886.28,
        933.33,
        980.17,
        956.75,
    ],
    dtype=np.double,
)

num_branch_geometry_nodes = np.array([len(geometry_nodes_x)], dtype=np.int32)

### Instantiate UGridNetwork1D class

In [10]:
network1d = UGridNetwork1D(
    name="network",
    node_x=node_x,
    node_y=node_y,
    edge_node=branch_node,
    edge_length=branch_length,
    edge_order=branch_order,
    geometry_nodes_x=geometry_nodes_x,
    geometry_nodes_y=geometry_nodes_y,
    num_edge_geometry_nodes=num_branch_geometry_nodes,
)

### Write the UGridNetwork1D instance to file and add crs and conventions

In [11]:
with UGrid("Network1DWrite.nc", "w+") as ug:
    # 1. Define a new network1d
    topology_id = ug.network1d_define(network1d)
    # 2. Write network1d
    ug.network1d_put(topology_id, network1d)
    # 3. Add crs to file
    attribute_dict = {
        "name": "Unknown projected",
        "epsg": np.array([0], dtype=int),
        "grid_mapping_name": "Unknown projected",
        "longitude_of_prime_meridian": np.array([0.0], dtype=float),
        "semi_major_axis": np.array([6378137.0], dtype=float),
        "semi_minor_axis": np.array([6356752.314245], dtype=float),
        "inverse_flattening": np.array([6356752.314245], dtype=float),
        "EPSG_code": "EPSG:0",
        "value": "value is equal to EPSG code"}
    ug.variable_int_with_attributes_define("projected_coordinate_system", attribute_dict)
    # 4. Add conventions (global attributes)
    conventions = {
        "institution": "Deltares",
        "references": "Unknown",
        "source": "Unknown Unknown. Model: Unknown",
        "history": "Created on 2017-11-27T18:05:09+0100, Unknown",
        "Conventions": "CF-1.6 UGRID-1.0/Deltares-0.8"}
    ug.attribute_global_define(conventions)